In [1]:
import pandas as pd
import json
import numpy as np
from time import sleep
import time
from tqdm import tqdm
import requests
from pydantic import BaseModel
url = open("local_models_path.txt", "r").read()

In [2]:
tests = ['piqa-en', 'piqa-mk', 'piqa-bg', 'piqa-sl', 'piqa-sr_cyrl', 'piqa-hr', 'piqa-sr_latn', 'piqa-bs', 'piqa-sl-cer', 'piqa-hr-ckm']

In [3]:
def run_local_model(model, prompt, url=url):

	class ReponseStructure(BaseModel):
		answer: int

	data = {
	    "model": model,
	    "prompt": prompt,
	    "stream": False,
	    "temperature": 0,
	    "format": ReponseStructure.model_json_schema()
	}

	headers = {"Content-Type": "application/json",}
	response = requests.post(url, json=data, headers=headers)

	return response.json()["response"]

In [4]:
models = ["gemma3:27b", "llama3.3:latest", "qwen3:32b", "deepseek-r1:14b"]

In [5]:
def predict_gpt(df_test_name, gpt_model):

	df_path = f"../../datasets/{df_test_name}.jsonl"

	responses = []
	instance_number = 0

	start_time = time.time()
	for line in open(df_path):
		instance_number += 1
		entry=json.loads(line)

		prompt= f"""
		### Task
			Given the following situation, which option is more likely to be correct?

			Situation: {entry['prompt']}

			Option 0: {entry['solution0']}

			Option 1: {entry['solution1']}
				
		### Output format
			Return a valid JSON dictionary with the following key: 'answer' and a value should be either 0 (if option 0 is more plausible) or 1 (if option 1 is more plausible).
		"""

		if gpt_model == "GaMS-27B":
			gpt_model_path = "hf.co/mradermacher/GaMS-27B-Instruct-i1-GGUF:i1-Q4_K_M"
		else:
			gpt_model_path = gpt_model

		initial_response= run_local_model(gpt_model_path, prompt, url=url)

		response = initial_response.replace("\n", "")
		response = response.replace("\t", "")

		# Get out a label
		try:
			# Convert the string into a dictionary
			response = json.loads(response)
			predicted = response["answer"]
			responses.append(predicted)
		# add a possibility of something going wrong
		except:
			predicted = initial_response
			print("error with extracting a label:")
			print(initial_response)
			responses.append(initial_response)

	end_time = time.time()
	elapsed_time_min = end_time-start_time

	print(f"Prediction finished. It took {elapsed_time_min/60} min for {instance_number} instances - {elapsed_time_min/instance_number} s per instance.")

	# Create a json with results

	current_results = {
		"system": gpt_model,
		"predictions": [
			{
			"train": "NA (zero-shot)",
			"test": "{}".format(df_test_name),
			"predictions": responses,
			},
		]
		}

	# Save the results as a new json
	with open("submissions/submission-{}-{}.json".format(gpt_model, df_test_name), "w") as file:
		json.dump(current_results, file)

	print("Classification with {} on {} finished.".format(gpt_model, df_test_name))

In [ ]:
# First, check that everything works well by evaluating the models on English
for model in models:
	print(model)
	predict_gpt("piqa-en", model)

gemma3:27b


In [ ]:
tests = ['piqa-en', 'piqa-mk', 'piqa-bg', 'piqa-sl', 'piqa-sr_cyrl', 'piqa-hr', 'piqa-sr_latn', 'piqa-bs', 'piqa-sl-cer', 'piqa-hr-ckm']

In [ ]:
# Now, evaluate the models on all other datasets
for test in tests[1:]:
	for model in models:
		print(model)
		predict_gpt(test, model)
